<a href="https://colab.research.google.com/github/gtrofimiuk/ML_course/blob/HW1/MusicGenres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Датасет https://www.kaggle.com/datasets/vicsuperman/prediction-of-music-genre/data

Датасет есть в репозитории, его надо положить в папку sample_data/prediction-of-music-genre

In [1]:
import os
os.makedirs("sample_data/prediction-of-music-genre", 0o777, exist_ok=True)

In [2]:
!pip install basemap > None

In [3]:
!pip install --upgrade matplotlib > None # need refresh enviroment after installation

In [47]:
!pip install feature_engine > None

In [5]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import feature_engine

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.cbook import boxplot_stats
from mpl_toolkits.basemap import Basemap

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA

%matplotlib inline

# Данные

Набор данных состоит из различных параметров музыкальных треков

In [20]:
dataset_path = "sample_data/prediction-of-music-genre/music_genre.csv"
data = pd.DataFrame(pd.read_csv(dataset_path))

In [21]:
data.head()

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,4-Apr,0.759,Electronic
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,4-Apr,0.531,Electronic
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,4-Apr,0.270,Electronic
4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,4-Apr,0.323,Electronic


In [22]:
data.shape

(50005, 18)

## Признаки в датасете

In [23]:
data.dtypes

instance_id         float64
artist_name          object
track_name           object
popularity          float64
acousticness        float64
danceability        float64
duration_ms         float64
energy              float64
instrumentalness    float64
key                  object
liveness            float64
loudness            float64
mode                 object
speechiness         float64
tempo                object
obtained_date        object
valence             float64
music_genre          object
dtype: object

Как можно увидеть, изначально у нас есть 18 описание которых можно найти на https://developer.spotify.com/documentation/web-api/reference/get-audio-features

Ниже курсивом буду писать описание из этого сайта

(Отмечу, что на данный момент Spotify запрещает использовать свое API для тренировки моделей машинного обучения, но, в РФ они не работают, да и датасет лежит в открытом доступе, так что ладно. Но вот обновить его уже сложно)

* instance_id - Идентификатор, без особого смысла
* artist_name - Исполнитель песни
* track_name - Название трека
* popularity - *The popularity of the track. The value will be between 0 and 100, with 100 being the most popular*

 *The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are.
Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past*
* acousticness - *A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic*

* danceability - *Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable*

* duration_ms - *The duration of the track in milliseconds*

* energy - *Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy*

* instrumentalness - *Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0*

* key - *The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.*

  В русской музыкальной традции ближайший аналог - тональность

* liveness - *Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.*

  Сомнительно, что это можно использовать для классификации, но посмотрим. Возможно, что есть жанры, где чаще записывают лайвы

* loudness - *The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db*

  Тоже вопрос, как эта метрика измерялась. Наверное, некоторые жанры такой признак может различить, поскольку электро обычно сильно громче блюза. Но, конечно же, многое зависит от сведения и мастеринга у трека

* mode - *Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0*

  Ближайший аналог - лад, но, лично я никогда не видел, чтобы key и mode рассматривались отдельно

* speechiness - *Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks*

  Тут конечно некоторые жанры можно сильно различить. Например, рэп от классической музыки

* tempo - *The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration*

* obtained_date - дата извлечения из датасета, тоже смысла не имеет

* valence - *A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry)*

* music_genre - Жанр музыки. Вообще, если бы это была серьезная работа для индустрии, то надо было бы заморочиться с тем, что такое жанр и как мы отнесли имеющиеся композиции к разным жанрам. Но, сейчас, в учебных целях, будем считать что Spotify разметил данные вот так и будем предсказывать по этой модели


**target: music_genre**


In [24]:
df = data.drop('instance_id', axis=1)
df.drop('obtained_date', inplace=True, axis=1)

In [25]:
df.head()

,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,music_genre
0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,0.759,Electronic
1,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,0.531,Electronic
2,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,0.333,Electronic
3,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,0.270,Electronic
4,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,0.323,Electronic


In [26]:
df.shape

(50005, 16)

In [27]:
data.isna().sum()

instance_id         5
artist_name         5
track_name          5
popularity          5
acousticness        5
danceability        5
duration_ms         5
energy              5
instrumentalness    5
key                 5
liveness            5
loudness            5
mode                5
speechiness         5
tempo               5
obtained_date       5
valence             5
music_genre         5
dtype: int64

In [28]:
df = df.dropna()

# Кодирование категориальных данных



## Тональность

По своей сути, тональность (key) имеет циклическую структру, более того, круги можно сделать даже разные

*   Хроматический круг - берем ноты по порядку: C, C#, D, D#,...
*   Кварто-квинтовый круг для мажора - C, G, D, A, E, B,...
*   Кварто-квинтовый круг для минора - A, E, B, F#, C#

По сути, тут кодирование такое же, как и у времени
https://stats.stackexchange.com/questions/311494/best-practice-for-encoding-datetime-in-machine-learning

In [29]:
Chromatic = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]
FifthsMajor = ["C","G","D","A","E","B","F#","C#","G#","D#","A#","F"]
FifthsMinor = ["A","E","B","F#","C#","G#","D#","A#","F","C","G","D"]

def GetIntegerMapFromList(InputList):
  res = dict()
  for i in range(len(InputList)):
    res[InputList[i]] = i
  return res

ChromaticMap = GetIntegerMapFromList(Chromatic)
FifthsMajorMap = GetIntegerMapFromList(FifthsMajor)
FifthsMinorMap = GetIntegerMapFromList(FifthsMinor)

print(ChromaticMap, FifthsMajorMap, FifthsMinorMap)

{'C': 0, 'C#': 1, 'D': 2, 'D#': 3, 'E': 4, 'F': 5, 'F#': 6, 'G': 7, 'G#': 8, 'A': 9, 'A#': 10, 'B': 11} {'C': 0, 'G': 1, 'D': 2, 'A': 3, 'E': 4, 'B': 5, 'F#': 6, 'C#': 7, 'G#': 8, 'D#': 9, 'A#': 10, 'F': 11} {'A': 0, 'E': 1, 'B': 2, 'F#': 3, 'C#': 4, 'G#': 5, 'D#': 6, 'A#': 7, 'F': 8, 'C': 9, 'G': 10, 'D': 11}


In [30]:
df.insert(df.columns.get_loc("key")+1, "key_chromatic", 0)
df.insert(df.columns.get_loc("key_chromatic")+1, "key_fifths", 0)

In [31]:
df['key_chromatic'] = df['key'].apply(lambda x: ChromaticMap[x])
#мажор минор тоже закодируем в 1 и 0, соответственно
df['mode'] = df['mode'].apply(lambda x: 1 if x == "Major" else 0)
df['key_fifths'] = df.apply(lambda x: FifthsMajorMap[x['key']] if x['mode'] == 1 else FifthsMinorMap[x['key']], axis=1)

In [38]:
from feature_engine.creation import CyclicalFeatures
cf = CyclicalFeatures()
cf.drop_original = True
index = df.columns.get_loc("key")
kc_fit = cf.fit_transform(pd.DataFrame(df["key_chromatic"]))
kf_fit = cf.fit_transform(pd.DataFrame(df["key_fifths"]))
kf_fit.head()

,key_fifths_sin,key_fifths_cos
0,-7.557496e-01,-0.654861
1,-2.449294e-16,1.000000
2,-9.898214e-01,-0.142315
3,-7.557496e-01,-0.654861
4,-2.817326e-01,-0.959493


In [41]:
#Добавляем по два новых столбца

index = df.columns.get_loc("key_chromatic")+1
for col in kc_fit.columns:
  df.insert(index, col, kc_fit[col])
  index += 1

index = df.columns.get_loc("key_fifths")+1
for col in kf_fit.columns:
  df.insert(index, col, kf_fit[col])
  index += 1

In [ ]:
df.drop('key', inplace=True, axis=1)
df.drop('key_chromatic', inplace=True, axis=1)
df.drop('key_fifths', inplace=True, axis=1)

In [46]:
df.head()

,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key_chromatic_sin,key_chromatic_cos,key_fifths_sin,key_fifths_cos,liveness,loudness,mode,speechiness,tempo,valence,music_genre
0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,-0.540641,0.841254,-7.557496e-01,-0.654861,0.115,-5.201,0,0.0748,100.889,0.759,Electronic
1,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,0.909632,0.415415,-2.449294e-16,1.000000,0.124,-7.043,0,0.0300,115.00200000000001,0.531,Electronic
2,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,-0.989821,-0.142315,-9.898214e-01,-0.142315,0.534,-4.617,1,0.0345,127.994,0.333,Electronic
3,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,0.540641,0.841254,-7.557496e-01,-0.654861,0.157,-4.498,1,0.2390,128.014,0.270,Electronic
4,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,-0.281733,-0.959493,-2.817326e-01,-0.959493,0.157,-6.266,1,0.0413,145.036,0.323,Electronic


## Жанры

Разныз жанров немного, поэтому начнем с OneHot Encoding

## Конструирование признаков

Я все же вижу смысл модели посмотреть на элементарные признаки из текста.

Количество слов у исполнителя и песни, количество символов в тексте песни и исполнителя
Наличие скобок в тексте песни
